In [1]:
import json
import pandas as pd
from tqdm import tqdm
from openai import OpenAI, AzureOpenAI

In [2]:
client = AzureOpenAI(
            azure_endpoint="https://cullmsouthindia.openai.azure.com/",
            api_key="037155e1b16a432fa836637370eca0e3",
            api_version="2024-02-15-preview",
        )

In [5]:
dataset = json.load(open("cleaned_final_modeLing.json",'r'))
dataset[0]

{'name': 'Abun',
 'type': ['POSS'],
 'difficulty': 5,
 'data': ["Abun: gap sye bi gan\nEnglish: big rat's offspring",
  'Abun: ji bi ngwe\nEnglish: my garden',
  'Abun: ji syim\nEnglish: my arm',
  "Abun: ndar sye gwes\nEnglish: big dog's leg",
  "Abun: gap bi ngwe\nEnglish: rat's garden"],
 'questions': ['Abun: ji gwes',
  'Abun: ji bi gan',
  'English: my big garden',
  "English: dog's offspring"],
 'answers': ['English: my leg',
  'English: my offspring',
  'Abun: ji bi ngwe sye',
  'Abun: ndar bi gan'],
 'cleaned_data': {'data': [[['Abun', 'gap sye bi gan'],
    ['English', "big rat's offspring"]],
   [['Abun', 'ji bi ngwe'], ['English', 'my garden']],
   [['Abun', 'ji syim'], ['English', 'my arm']],
   [['Abun', 'ndar sye gwes'], ['English', "big dog's leg"]],
   [['Abun', 'gap bi ngwe'], ['English', "rat's garden"]]],
  'qna': [[['Abun', 'ji gwes'], ['English', 'my leg']],
   [['Abun', 'ji bi gan'], ['English', 'my offspring']],
   [['English', 'my big garden'], ['Abun', 'ji bi n

In [26]:
def question_data_to_puzzle_prompt(core_question_data):
    BASE_PROMPT = """Here are some expressions in <LANG> (a never-seen-before foreign language) and their translations
in English:

<DATA>

Given the above examples, please translate the following statements.

<QUESTIONS>

"""

    data_pairs = []
    for pair in core_question_data["data"]:
        data_pairs.append(f"\t\"{pair[0][0]}\":\"{pair[0][1]}\"\n\t\"{pair[1][0]}\":\"{pair[1][1]}\"")
    
    question_pairs = []
    for pair in core_question_data["qna"]:
        question_pairs.append(f"\t\"{pair[0][0]}\":\"{pair[0][1]}\"")

    return BASE_PROMPT.replace("<LANG>",core_question_data["data"][0][0][0]).replace("<DATA>",',\n\n'.join(data_pairs)).replace("<QUESTIONS>",',\n'.join(question_pairs))


print(question_data_to_puzzle_prompt(dataset[0]["cleaned_data"]))

Here are some expressions in Abun (a never-seen-before foreign language) and their translations
in English:

	"Abun":"gap sye bi gan"
	"English":"big rat's offspring",

	"Abun":"ji bi ngwe"
	"English":"my garden",

	"Abun":"ji syim"
	"English":"my arm",

	"Abun":"ndar sye gwes"
	"English":"big dog's leg",

	"Abun":"gap bi ngwe"
	"English":"rat's garden"

Given the above examples, please translate the following statements.

	"Abun":"ji gwes",
	"Abun":"ji bi gan",
	"English":"my big garden",
	"English":"dog's offspring"




In [18]:
dataset[0]["cleaned_data"]

{'data': [[['Abun', 'gap sye bi gan'], ['English', "big rat's offspring"]],
  [['Abun', 'ji bi ngwe'], ['English', 'my garden']],
  [['Abun', 'ji syim'], ['English', 'my arm']],
  [['Abun', 'ndar sye gwes'], ['English', "big dog's leg"]],
  [['Abun', 'gap bi ngwe'], ['English', "rat's garden"]]],
 'qna': [[['Abun', 'ji gwes'], ['English', 'my leg']],
  [['Abun', 'ji bi gan'], ['English', 'my offspring']],
  [['English', 'my big garden'], ['Abun', 'ji bi ngwe sye']],
  [['English', "dog's offspring"], ['Abun', 'ndar bi gan']]]}

In [19]:
question_data_to_puzzle_prompt(dataset[0]["cleaned_data"])

'Here are some expressions in Abun (a never-seen-before foreign language) and their translations\nin English:\n\n\t"Abun":"gap sye bi gan"\n\t"English":"big rat\'s offspring",\n\n\t"Abun":"ji bi ngwe"\n\t"English":"my garden",\n\n\t"Abun":"ji syim"\n\t"English":"my arm",\n\n\t"Abun":"ndar sye gwes"\n\t"English":"big dog\'s leg",\n\n\t"Abun":"gap bi ngwe"\n\t"English":"rat\'s garden"\n\nGiven the above examples, please translate the following statements.\n\n\t"Abun":"ji gwes",\n\t"Abun":"ji bi gan",\n\t"English":"my big garden",\n\t"English":"dog\'s offspring"\n\n'

In [8]:
def fetch_solution_text(core_question_data):

    prompt = question_data_to_puzzle_prompt(core_question_data)
    message = [
    {"role": "system", "content": "You are a helpful assistant."},
    # {"role": "system", "content": "You are a helpful assistant. Response only using valid JSON objects."},
    {
        "role": "user",
        "content": prompt,
    },
    ]

    # print(message)

    max_retry = 3
    while max_retry > 0:
        try:
            response = client.chat.completions.create(
                model="gpt4",
                messages=message,
                temperature=0,
                max_tokens=2048,
                # response_format={"type": "json_object"},
                seed=7777
            )

            result = response.choices[0].message.content.strip()
            return result
        except:
            print("error in JSONDecodeError")
            max_retry -= 1
    return ""

# fetch_solution_text(dataset[0]["cleaned_data"])

In [13]:
fetch_solution_text(dataset[0]["cleaned_data"])

'Based on the provided examples, we can deduce the meanings of individual words in Abun and their English equivalents:\n\n- "ji" seems to mean "my"\n- "bi" seems to be a possessive marker, similar to "\'s" in English\n- "sye" seems to mean "big"\n- "gan" seems to mean "offspring"\n- "ngwe" seems to mean "garden"\n- "gwes" seems to mean "leg", but in the context of "ndar sye gwes" translating to "big dog\'s leg", "ndar" likely means "dog" and "gwes" could be contextually inferred as "dog\'s leg" or simply "dog"\n\nGiven this information, let\'s translate the provided Abun statements:\n\n1. "Abun":"ji gwes"\n   Since "ji" means "my" and "gwes" in the context of "ndar sye gwes" seems to be related to "dog", but we don\'t have a direct translation for "gwes" alone, it\'s a bit ambiguous. However, if we assume that "gwes" retains its meaning related to "dog" from the previous example, the translation could be "my dog". But since we don\'t have a direct translation for "gwes" alone, this is 

In [9]:
def fetch_final_answer_JSON(core_question_data, auto_solution):

    # Set as "INVALID" if no valid answer is detected.
    pairs = []
    for pair in core_question_data["qna"]:

        pairs.append("""
        {
            "<LANG0>": "<ANS0>",
            "<LANG1>": "" # Corresponding answer from solution.
        }
""".replace("<LANG0>",pair[0][0]).replace("<LANG1>",pair[1][0]).replace("<ANS0>",pair[0][1]))
        
        json_format = "{\n\t\"answers\": [\n" + ',\n'.join(pairs) + "\t]\n}"

    core_problem_prompt = question_data_to_puzzle_prompt(core_question_data)

    message_text = [
    # {"role": "system", "content": "You are a helpful assistant."},
    {"role": "system", "content": "You are a helpful assistant. Respond only using valid JSON objects."},
    {
        "role": "user",
        "content": """Observe the following linguistics puzzle.

--- puzzle start ---
<CORE_PROBLEM>
--- puzzle end ---

Here's a possible solution to the puzzle. 

--- solution start ---
<AUTO_SOLUTION>
--- solution end ---

Extract the final answers for the questions listed in the puzzle from the solution as a valid JSON in the following format:

--- JSON format start ---
<JSON_FORMAT>
--- JSON format end ---

Do not try to solve the puzzle yourself or modify the answers from the given solution in any way.
""".replace("<CORE_PROBLEM>",core_problem_prompt).replace("<AUTO_SOLUTION>",auto_solution).replace("<JSON_FORMAT>",json_format),
    },
    ]

    # print(message_text[1]["content"])

    max_retry = 3
    while max_retry > 0:
        try:
            response = client.chat.completions.create(
                model="gpt4",
                messages=message_text,
                temperature=0,
                max_tokens=2048,
                response_format={"type": "json_object"},
                seed=7777
            )

            result = json.loads(response.choices[0].message.content)
            return result
        except:
            print("error in JSONDecodeError")
            max_retry -= 1
    return ""

In [22]:
dataset[1]["cleaned_data"]

{'data': [[['Ainu', 'Numan hambaga ku=e.'],
   ['English', 'Yesterday I ate a hamburger.']],
  [['Ainu', 'Ku=kor acapo pirka nispa ne.'],
   ['English', 'My uncle is a good chief.']],
  [['Ainu', 'Husko cise e=kor.'], ['English', 'You have an old house.']],
  [['Ainu', 'E=pake sonno poro.'], ['English', 'Your head is very big.']],
  [['Ainu', 'Ukran ku=kor hapo kimunkamuy nukar.'],
   ['English', 'Last night my mother saw a bear.']],
  [['Ainu', 'Tan pone e=oterke.'], ['English', 'You step on this bone.']],
  [['Ainu', 'Ku=cikir tanne.'], ['English', 'My leg is long.']],
  [['Ainu', 'Sita ermun kuykuy.'], ['English', 'The dog bites a mouse.']],
  [['Ainu', 'Wakka wa cep terke.'],
   ['English', 'The fish jumps from the water.']],
  [['Ainu', 'Hoska=numan Nupurpet en ku=oowet.'],
   ['English', 'Two days ago I returned to Noboribetsu.']]],
 'qna': [[['Ainu', 'Tan kimunkamuy poro cep e.'],
   ['English', 'This bear ate a big fish.']],
  [['Ainu', 'E=kor hapo sonno husko.'],
   ['English'

In [10]:
solutions = []
answers = []
for d in tqdm(dataset):
    s = fetch_solution_text(d["cleaned_data"])
    j = fetch_final_answer_JSON(d["cleaned_data"], s)
    solutions.append(s)
    answers.append(j)

100%|██████████| 48/48 [12:32<00:00, 15.68s/it]


In [11]:
answers

[{'answers': [{'Abun': 'ji gwes', 'English': 'my leg'},
   {'Abun': 'ji bi gan', 'English': 'my offspring'},
   {'English': 'my big garden', 'Abun': 'ji sye ngwe'},
   {'English': "dog's offspring", 'Abun': 'ndar bi gan'}]},
 {'answers': [{'Ainu': 'Tan kimunkamuy poro cep e.',
    'English': 'The bear is very big and jumps.'},
   {'Ainu': 'E=kor hapo sonno husko.',
    'English': 'Your mother has a very old house.'},
   {'Ainu': 'Ukran kangaruu ku=cikir kuykuy.',
    'English': 'Last night a kangaroo bit my leg.'},
   {'Ainu': 'E=kuykuy pone sonno husko.',
    'English': 'The mouse steps on a very old house.'},
   {'English': 'The mouse that I saw ate a big hamburger.',
    'Ainu': 'Ku=oowet kuykuy sonno poro numan hambaga.'},
   {'English': 'The old dog has a good head.',
    'Ainu': 'Husko seta pirka e=pake.'},
   {'English': 'The big bear is a mother.',
    'Ainu': 'Kimunkamuy poro hapo ku=kor.'},
   {'English': 'Two days ago my uncle returned from the water to the old house.',
    

In [12]:
solutions

['Based on the provided examples, we can try to deduce the meanings of individual words in the Abun language and then apply this knowledge to translate the new statements.\n\nFrom the examples given:\n\n- "ji" seems to mean "my"\n- "bi" seems to be a possessive marker, similar to "\'s" in English\n- "sye" seems to mean "big"\n- "gan" seems to mean "offspring"\n- "ngwe" seems to mean "garden"\n- "gwes" seems to mean "leg", but in the context of "ndar sye gwes" translating to "big dog\'s leg", "ndar" likely means "dog" and "gwes" could be inferred as "leg" by itself or "dog\'s leg" as a compound. However, since "bi" is the possessive marker, "gwes" should stand for "leg".\n\nNow, let\'s translate the new statements:\n\n"Abun":"ji gwes"\n- "ji" means "my"\n- "gwes" means "leg"\nSo, "ji gwes" translates to "my leg".\n\n"Abun":"ji bi gan"\n- "ji" means "my"\n- "bi" is the possessive marker\n- "gan" means "offspring"\nSo, "ji bi gan" translates to "my offspring".\n\nThe provided English tran

In [24]:
## MERGE
res = []
for ridx, (d, s, a) in enumerate(zip(dataset, solutions, answers)):
    _row = {
        "problem_name": d["name"],
        "type": d["type"][0],
        "difficulty": d["difficulty"],
        "data": json.dumps(d["cleaned_data"]["data"]),
        "solution_auto": s,
    }
    try:
        for idx in range(len(d["cleaned_data"]["qna"])):
            row = _row.copy()
            row["source_lang"] = d["cleaned_data"]["qna"][idx][0][0]
            row["source_sentence"] = d["cleaned_data"]["qna"][idx][0][1]
            row["target_lang"] = d["cleaned_data"]["qna"][idx][1][0]
            row["target_sentence_gold"] = d["cleaned_data"]["qna"][idx][1][1]
            row["target_sentence_auto"] = a["answers"][idx][row["target_lang"]]
            res.append(row)
    except:
        print(ridx)
        raise Exception()

res_df = pd.DataFrame(res)

In [25]:
res_df.head()

,problem_name,type,difficulty,data,solution_auto,source_lang,source_sentence,target_lang,target_sentence_gold,target_sentence_auto
0,Abun,POSS,5,"[[[""Abun"", ""gap sye bi gan""], [""English"", ""big...","Based on the provided examples, we can try to ...",Abun,ji gwes,English,my leg,my leg
1,Abun,POSS,5,"[[[""Abun"", ""gap sye bi gan""], [""English"", ""big...","Based on the provided examples, we can try to ...",Abun,ji bi gan,English,my offspring,my offspring
2,Abun,POSS,5,"[[[""Abun"", ""gap sye bi gan""], [""English"", ""big...","Based on the provided examples, we can try to ...",English,my big garden,Abun,ji bi ngwe sye,ji sye ngwe
3,Abun,POSS,5,"[[[""Abun"", ""gap sye bi gan""], [""English"", ""big...","Based on the provided examples, we can try to ...",English,dog's offspring,Abun,ndar bi gan,ndar bi gan
4,Ainu,ORDER,5,"[[[""Ainu"", ""Numan hambaga ku=e.""], [""English"",...",Based on the provided Ainu to English translat...,Ainu,Tan kimunkamuy poro cep e.,English,This bear ate a big fish.,The bear is very big and jumps.


In [10]:
res_df.to_csv("ModeLing_baseline_eval_v3_gpt4.csv",index=False)

In [11]:
res_df["type"].unique()

array(['POSS', 'ORDER', 'NOUN-ADJ', 'SEM'], dtype=object)

In [12]:
## EXACT MATCH
res_df["score_exact_match"] = [g.strip()==a.strip() for g, a in zip(res_df["target_sentence_gold"],res_df["target_sentence_auto"])]

In [13]:
print(res_df["score_exact_match"].mean())

0.5477941176470589
